# FYP Project 2 Group G

In [5]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import Config
import os, sys 
from scipy.stats import pearsonr, spearmanr 
import seaborn as sns
import skimage.io as sk
from skimage.color import rgb2gray
from statsmodels.robust import mad
from PIL import Image 
import ptitprince as pt 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

SyntaxError: invalid syntax (<ipython-input-5-24c97328c9d1>, line 9)

In [ ]:
path_image = 'data/example_image'
path_mask = '../data/example_segmentation'


df = pd.read_csv(Config.example_ground_truth_path)

features_df = pd.read_csv(Config.features_path)
image_id = list(df['image_id'])



In [ ]:
def get_assymetry(image):
    ''' get the assymetry for a given mask by folding it in half from multiple angles'''
    return round(np.mean([assymetry(image), assymetry(image.rotate(30, expand= True)),assymetry(image.rotate(60, expand= True)),assymetry(image.rotate(90, expand= True))]),2)

In [ ]:
def assymetry(image):
    ''' get the assymetry between the left and right part of a given binary mask '''
    width, height = image.size #mask should be quadratic and therefore have equal dimension
    size = width * height

    #check for uneven number of pixels (should not happen but just as a precaution)
    if width %2 != 0:
        raise TypeError("Uneven number of pixel and cannot be symmetric")
    
    #cut in half and fold
    left = image.crop((0, 0, (width/2), height)) #left part of picture (left, top, right, bottom)
    right = image.crop(((width/2), 0, width, height)) #right part of picture
    right = right.transpose(Image.FLIP_LEFT_RIGHT) #flip right part to compare

    #get the binary difference between left an right
    symmetry = np.where(np.array(left) != np.array(right), 1, 0)

    return np.sum(symmetry) / (size/2) #the percentage of assymetry 
    

In [ ]:
def cut_image(image):
    width, height = image.size
    image = np.array(image)
    
    if width %2 != 0:
        image = np.delete(image, -1, 1)

    if height %2 != 0:
        image = np.delete(image, -1, 0)

    image = Image.fromarray(image)

    return image

In [ ]:
num_images = len(image_id)

assymetry_values = []

#Loop through all images
for i in np.arange(num_images):
    
    # Define filenames related to this image
    file_image = path_image + os.sep + image_id[i] + '.jpg'
    file_mask = path_mask + os.sep + image_id[i] + '_segmentation.png'
    
    # Read the images with these filenames
    image_file = Image.open(file_mask) # open mask image
    image_file = cut_image(image_file)
    # im = plt.imread(file_image)
    # mask = plt.imread(file_mask)
    # image_file = Image.open(file_mask) # open mask image (for assymetry we need image.open and not plt)
    
    # # Measure features (custom made function)
    # a, p = util.measure_area_perimeter(mask)
    
    # # Store in the variables we created before
    # features_area[i,0] = a
    # features_perimeter[i,0] = p
    
    """checking for assymetry"""
    assymetry_values.append([image_id[i],get_assymetry(image_file)])

In [ ]:
def compactness(p, A):
    """ Computes and returns the compactness of a figure """
    return   round(4*np.pi*A / p ** 2, 4)

In [ ]:
compactlst = list()
for p, A in zip(features_df['perimeter'], features_df['area']):
    compactlst.append(compactness(p, A))

In [ ]:
df['compactness'] = compactlst

In [ ]:
assymetry_df = pd.DataFrame(assymetry_values, columns=["name", "assymetry"])
merged =df.merge(assymetry_df,left_on= "image_id", right_on = "name")

merged.drop(["name", "seborrheic_keratosis"], axis=1)


,image_id,melanoma,compactness,assymetry
0,ISIC_0001769,0.0,0.6703,0.03
1,ISIC_0001852,0.0,0.8711,0.01
2,ISIC_0001871,0.0,0.8713,0.03
3,ISIC_0003462,0.0,1.1250,0.03
4,ISIC_0003539,0.0,0.9361,0.06
...,...,...,...,...
145,ISIC_0015443,0.0,0.6446,0.06
146,ISIC_0015445,0.0,0.3975,0.38
147,ISIC_0015483,0.0,0.5833,0.09
148,ISIC_0015496,0.0,0.5957,0.12


## PLOTS

In [ ]:
#creating a lineplot of the distribution of the assymetry values
for_plot = np.unique(merged["assymetry"], return_counts=True)
sns.lineplot(x=for_plot[0], y=for_plot[1])

In [ ]:
#create a scatterplot that shows the assymetry on the y axis 
#melanoma are colored red, benign lesions are colored green
colors1 = ['#8BA888','#D67573']
colors = ["green" if merged['melanoma'][x] == 0.0 else 'red' for x in range(len(merged['melanoma'])) ]

sns.scatterplot(x = [i for i in range(len(merged["assymetry"]))],y = merged["assymetry"], hue= colors, palette=colors1)

In [ ]:
#create a rain cloud plot that plots the assymetry distribution for the two different groups (melanoma, benign)
sns.set(style="whitegrid",font_scale=1.5)

dx = list(merged["melanoma"]); dy = list(merged["assymetry"])
f, ax = plt.subplots(figsize = (7,5))
sigma = .2; ort = "h"

ax = pt.RainCloud(x = dx, y = dy, width_viol = .6, ax = ax, bw = sigma, orient= ort)
plt.title("Raincloud plot of the melanoma assymetry")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=932aa97b-f6d2-4889-8f04-d6166675cc2f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>